If looking for the demo using Jordan's data, go to camera_functionality_example.ipynb

In [1]:
import pandas as pd

In [2]:
path = '/Users/jasmineschoch/Documents/UW/VBL/urchin-examples/data/run_2023-02-24/'
files = pd.read_csv(path + 'file_names.csv', header=None)

,0
0,JRS_00042021-06-16_AudiocoordArray.csv
1,JRS_00042021-06-16_AudiocoordArrayFiltered.csv
2,JRS_00042021-06-16_LickscoordArray.csv
3,JRS_00042021-06-16_LickscoordArrayFiltered.csv
4,JRS_00042021-06-16_Visual StimulicoordArray.csv
...,...
59,JRS_00132022-04-28_LickscoordArrayFiltered.csv
60,JRS_00132022-04-28_Visual StimulicoordArray.csv
61,JRS_00132022-04-28_Visual StimulicoordArrayFil...
62,JRS_00132022-04-28coordArrayNonEncoding.csv


In [3]:
data = pd.DataFrame(columns=["x", "y", "z", "file"])

for i in range(len(files)):
    name = files.iloc[i, 0]
    try:
        df = pd.read_csv(path + name, header=None)
        df.columns = ["x", "y", "z"]
        df["file"] = name
        data = data.append(df, ignore_index=True)
        df.head()
    except pd.errors.EmptyDataError:
        #print("Empty file:", name)

Empty file: JRS_00052021-07-21_AudiocoordArrayFiltered.csv
Empty file: JRS_00052021-07-21_Visual StimulicoordArrayFiltered.csv
Empty file: JRS_00052021-08-16_Visual StimulicoordArrayFiltered.csv
Empty file: JRS_00132022-04-26_Visual StimulicoordArrayFiltered.csv
Empty file: JRS_00132022-04-27_Visual StimulicoordArray.csv
Empty file: JRS_00132022-04-27_Visual StimulicoordArrayFiltered.csv


In [4]:
#Adding feature labels to data
encoding = ['Visual', 'Licks', 'NonEncoding', 'Audio', "Wheel"]
labels = []

for file_name in data['file']:
    label = None
    
    for feature in encoding:
        if feature in file_name:
            label = feature
            break 
    
    labels.append(label)
data['labels'] = labels

In [5]:
#Adding colors
conditions = [
    data['labels'] == "Audio", #Yellow
    data['labels'] == "Licks", #Orange
    data['labels'] == "NonEncoding", #teal
    data['labels'] == "Visual", #light blue
    data['labels'] == "Wheel" #navy
]

choices = ["#F6D55C", "#ED553B", "#3CAEA3", "#20639B", "#173F5F"]

data['color'] = None  # Initialize the 'color' column with None

for i, condition in enumerate(conditions):
    data.loc[condition, 'color'] = choices[i]

In [6]:
data.head()

,x,y,z,file,labels,color
0,3854.099849,8700,3185.198183,JRS_00042021-06-16_AudiocoordArray.csv,Audio,#F6D55C
1,3854.099849,8700,3185.198183,JRS_00042021-06-16_AudiocoordArrayFiltered.csv,Audio,#F6D55C
2,3956.855249,8725.0,3788.986745,JRS_00042021-06-16_LickscoordArray.csv,Licks,#ED553B
3,3950.491288,8725.0,3744.439018,JRS_00042021-06-16_LickscoordArray.csv,Licks,#ED553B
4,3935.642046,8725.0,3640.494321,JRS_00042021-06-16_LickscoordArray.csv,Licks,#ED553B


In [7]:
import oursin as urchin
urchin.setup()
brain_areas = ["MRN", "SCs", "SCm", "APN", "RN"]

(URN) connected to server
Login sent with ID: root


In [8]:
#Generating overall brain image:
urchin.ccf.set_visibility({'grey':True})
urchin.ccf.set_material({'grey':'transparent-unlit'})
urchin.ccf.set_alpha({'grey':0.1})

In [9]:
#Generating brain area images:
for area in brain_areas:
    urchin.ccf.set_visibility({area:True})
    urchin.ccf.set_material({area:'transparent-unlit'})
    urchin.ccf.set_alpha({area:0.2})

In [10]:
#Creating neurons
neurons = []
for i in range(len(data)):
    neurons.append(str(i))
urchin.neurons.create(neurons)
#Setting neuron colors
neuron_colors = {}
for neuron in neurons:
  neuron_colors[neuron] = data.loc[int(neuron),"color"]
urchin.neurons.set_color(neuron_colors)
#Setting neuron positions
neuron_positions = {}
for neuron in neurons:
    neuron_positions[neuron] = [data.loc[int(neuron), "x"], data.loc[int(neuron), "y"], data.loc[int(neuron), "z"]]
urchin.neurons.set_position(neuron_positions)
